In [1]:
import pandas as pd
import re
import operator
import string
import nltk 
from nltk.tag import pos_tag

In [2]:
gg = pd.read_json('gg2013.json', orient='records')
gg1 = pd.read_json('gg2015.json', orient='records')

In [3]:
copy = gg.copy(deep=True)
copy.head(5)

,text,user,id,timestamp_ms
0,JLo's dress! #eredcarpet #GoldenGlobes,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}",290620657987887104,2013-01-14 00:45:38
1,What's making Sofia Vergara's boobs stay like ...,"{'screen_name': 'theAmberShow', 'id': 14648726}",290620657887219713,2013-01-14 00:45:38
2,RT @FabSugar: Kerry Washington is EVERYTHING. ...,"{'screen_name': 'SweetyPW', 'id': 35498686}",290620657828524032,2013-01-14 00:45:38
3,Anne Hathaway has got me living. #GoldenGlobes,"{'screen_name': '_NicoleEdwards', 'id': 144430...",290620657799159809,2013-01-14 00:45:38
4,Jennifer Lopez's lace dress? Thoughts? #Golden...,"{'screen_name': 'lolaogunnaike', 'id': 134953223}",290620657778188288,2013-01-14 00:45:38


In [4]:
def removePunc(x):
#     new_string = x.translate(str.maketrans('', '', string.punctuation))
    x = re.sub(r'[@#]\w+', '', x) #taking out hashtags and @ 
    x = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', '', x) #taking out links 
    x = re.sub(r'[!?\.,\'\"]+', '', x) #taking out punctuation i.e ? ! . ' and " 
    return x.strip()
# copy['text'] = copy['text'].str.lower()
copy['text'] = copy['text'].apply(lambda x: removePunc(str(x)))

In [5]:
test = list(copy['text'].values)
print(len(test))
s = list(set(test))
print(len(s))
df = pd.DataFrame(s, columns=['text'])

174643
123436


In [9]:
df = df.loc[(df['text'].str.contains('Best')) & (df['text'].str.contains('-'))]
# tes.to_csv('awards.csv', index=False)
df.head()

,text
89,Adele for Best Original Song - Skyfall
152,RT : 2013: Daniel Day-Lewis wins Best Actor i...
252,Best Television Series - Drama - Homeland () -
273,Best Screenplay - Motion Picture: Quentin Tara...
354,RT : Best Motion Picture - Comedy or Musical -...


In [ ]:
def what(text):
    tweet = text.split()
    ind = tweet.index('Best')
    if len(tweet) > 1:
        

In [7]:
def nGram(text):
    test = text
    firstsplit = test.split('Best', 1)
    if len(firstsplit) > 1:
        left = firstsplit[0].split(' ')
        left.reverse()
        right = firstsplit[1].split(' ')
        new_list = []
        x = ""
        for i in left:
            x =  i + x + ' '
            new_list.append(x)
        x = ""
        for i in right:
            x = x + i + ' '
            new_list.append(x)
        new_list = [x.rstrip() for x in new_list]
    else:
        new_list = []
    return new_list

In [8]:
FINAL_NGRAMS = []
NGRAMS_DICT = {}
for i in range(len(df)):
    
    l = nGram(df['text'].iloc[i])
    for j in l:
        if j in NGRAMS_DICT:
            NGRAMS_DICT[j] += 1
        else:
            NGRAMS_DICT[j] = 1
cd = sorted(NGRAMS_DICT.items(),key=operator.itemgetter(1),reverse=True)
cd

[('', 3110),
 (':', 393),
 ('RT:', 323),
 (' Actor', 244),
 (' Actress', 162),
 (' Motion', 157),
 (' Actor in', 154),
 (' Motion Picture', 153),
 (' Supporting', 150),
 (' Actor in a', 140),
 ('“:', 138),
 (' Motion Picture -', 127),
 (' Original', 122),
 ('wins', 121),
 ('for', 120),
 (' Actress in', 118),
 (' Actress in a', 110),
 (' Supporting Actress', 92),
 (' Director', 92),
 (' Actor in a Motion', 83),
 (' Actor in a Motion Picture', 82),
 (' Television', 75),
 (' Television Series', 75),
 (' Supporting Actress in', 73),
 (' Original Song', 71),
 (' Supporting Actress in a', 71),
 (' Director -', 68),
 (' Motion Picture - Drama', 67),
 (' TV', 59),
 (' Actor in a Motion Picture -', 56),
 (' Supporting Actor', 52),
 (' Supporting Actress in a Motion', 51),
 (' Screenplay', 50),
 (' Television Series -', 49),
 (' Supporting Actress in a Motion Picture', 49),
 (' Original Song -', 48),
 (' Supporting Actress in a Motion Picture -', 48),
 (' Actress in a Motion', 47),
 (' Motion Pi